# RAPIDS CuML
The RAPIDS library is now available in all Kaggle notebooks. Hooray! Simply type `import cuml` or `import cudf` to load the two most popular packages.

RAPIDS is described [here][1]. RAPIDS `cuDF` accelerates dataframe operations using GPU and has a similar api as Pandas. RAPIDS `cuML` accelerates machine learning algorithms using GPU and has a similar api as Scikit-Learn. Since RAPIDS ML algorithms are so fast, we can do things that were never possible like applying genetic algorithms to ML hyperparameter searchs!

[1]: https://rapids.ai/

# Load Libraries

In [ ]:
import sys, warnings
warnings.filterwarnings("ignore")
sys.path.append('../input/iterativestratification')

import pandas as pd, numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import log_loss

import cuml
print('RAPIDS',cuml.__version__)

# Load Data

In [ ]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
print('train shape',train.shape)
test = pd.read_csv('../input/lish-moa/test_features.csv')
print('test shape',test.shape)
targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

In [ ]:
train.cp_dose = train.cp_dose.map({'D1':-1,'D2':1})
test.cp_dose = test.cp_dose.map({'D1':-1,'D2':1})

train.cp_time = train.cp_time.map({24:-1, 48:0, 72:1})
test.cp_time = test.cp_time.map({24:-1, 48:0, 72:1})

train.cp_type = train.cp_type.map({'trt_cp':-1, 'ctl_vehicle':1})
test.cp_type = test.cp_type.map({'trt_cp':-1, 'ctl_vehicle':1})

train.head()

In [ ]:
target = targets['5-alpha_reductase_inhibitor']

In [ ]:
print(train.shape)
print(target.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Instantiate learning model (k = 3)
classifier = KNeighborsClassifier(n_neighbors=3)

In [ ]:
train.iloc[:,1:]
target

In [ ]:
import cudf, cuml
from cuml.neighbors import KNeighborsClassifier as cuKNeighbors
model = cuKNeighbors(n_neighbors=3)
model.fit(train.iloc[:,1:], target)

In [ ]:
X_test

In [ ]:
y_hat = model.predict(X_test.iloc[:, 1:])
y_hat